In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Dataset
---

In [ ]:
# ImageDataGenerator
# ------------------
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#directories used to handle the datasets
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')
training_dir = os.path.join(dataset_dir, 'training')
#validation_dir = os.path.join(dataset_dir, 'validation')
images_dir = os.path.join(training_dir, 'images', 'img')
masks_dir = os.path.join(training_dir, 'masks', 'img')

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(validation_split=0.1,
                                            rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='nearest',
                                            preprocessing_function=preprocess_input)
    train_mask_data_gen = ImageDataGenerator(validation_split=0.1,
                                             rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='nearest')
else:
    train_img_data_gen = ImageDataGenerator(validation_split=0.1,
                                            preprocessing_function=preprocess_input)
    train_mask_data_gen = ImageDataGenerator(validation_split=0.1)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

In [ ]:
# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       subset='training',
                                                       seed=SEED)  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         subset='training',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       subset='validation',
                                                       seed=SEED)
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         subset='validation',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)

In [ ]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))

def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), dtype=tf.float32)
    return x_, tf.where(y_ > 0, tf.ones_like(y_, dtype=tf.float32), tf.zeros_like(y_, dtype=tf.float32))

train_dataset = train_dataset.map(prepare_target)
    
# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 3]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
# Let's test data generator
# -------------------------
import time
import matplotlib.pyplot as plt

%matplotlib notebook

fig, ax = plt.subplots(1, 2)
fig.show()

# Assign a color to each class
colors_dict = {}
colors_dict[1] = [255, 255, 255]  # foreground
colors_dict[0] = [0, 0, 0]  # background

iterator = iter(train_dataset)

for _ in range(10):
    augmented_img, target = next(iterator)
    tf.shape(augmented_img)
    augmented_img = augmented_img[0]   # First element
    augmented_img = augmented_img * 255  # denormalize
    
    
    tf.shape(target)
    
    target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)
    
    # Assign colors (just for visualization)
    target_img = np.zeros([target.shape[0], target.shape[1], 3])
    
    target_img[np.where(target == 0)] = colors_dict[0]
    target_img[np.where(target == 1)] = colors_dict[1]
    
    ax[0].imshow(np.uint8(augmented_img))
    ax[1].imshow(np.uint8(target_img))
    
    fig.canvas.draw()
    #time.sleep(10)

In [ ]:
np.unique(target_img)
np.unique(target)

In [ ]:
"""Xception V1 model for Keras.
On ImageNet, this model gets to a top-1 validation accuracy of 0.790
and a top-5 validation accuracy of 0.945.
Do note that the input image format for this model is different than for
the VGG16 and ResNet models (299x299 instead of 224x224),
and that the input preprocessing function
is also different (same as Inception V3).
# Reference
- [Xception: Deep Learning with Depthwise Separable Convolutions](
    https://arxiv.org/abs/1610.02357) (CVPR 2017)
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

from tensorflow.keras import layers

TF_WEIGHTS_PATH_NO_TOP = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.4/'
    'xception_weights_tf_dim_ordering_tf_kernels_notop.h5')


def create_model():
    
    # Determine proper input shape
    input_shape = [256, 256, 3]

    img_input = layers.Input(shape=input_shape)

    channel_axis = -1 #channel last encoding

    #Block 1
    block1_conv1 = layers.Conv2D(32, (3, 3),
                      strides=(2, 2),
                      use_bias=False,
                      name='block1_conv1')(img_input)#127x127
    block1_conv1_bn = layers.BatchNormalization(axis=channel_axis, 
                                                name='block1_conv1_bn')(block1_conv1)
    block1_conv1_act = layers.Activation('relu', 
                                         name='block1_conv1_act')(block1_conv1_bn)
    block1_conv2 = layers.Conv2D(64, (3, 3), 
                                 use_bias=False, 
                                 name='block1_conv2')(block1_conv1_act)#125x125
    block1_conv2_bn = layers.BatchNormalization(axis=channel_axis, 
                                                name='block1_conv2_bn')(block1_conv2)
    block1_conv2_act = layers.Activation('relu', 
                                         name='block1_conv2_act')(block1_conv2_bn)

    block1_residual1 = layers.Conv2D(128, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(block1_conv2_act)#63x63
    block1_residual2 = layers.BatchNormalization(axis=channel_axis)(block1_residual1)

    #Block 2
    block2_sepconv1 = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv1')(block1_conv2_act)#125x125
    block2_sepconv1_bn = layers.BatchNormalization(axis=channel_axis, name='block2_sepconv1_bn')(block2_sepconv1)
    block2_sepconv2_act = layers.Activation('relu', name='block2_sepconv2_act')(block2_sepconv1_bn)
    block2_sepconv2 = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv2')(block2_sepconv2_act)#125x125
    block2_sepconv2_bn = layers.BatchNormalization(axis=channel_axis, name='block2_sepconv2_bn')(block2_sepconv2)

    block2_pool = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block2_pool')(block2_sepconv2_bn)#63x63
    block2_add = layers.add([block2_pool, block1_residual2])

    block2_residual1 = layers.Conv2D(256, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(block2_add)#32x32
    block2_residual2 = layers.BatchNormalization(axis=channel_axis)(block2_residual1)

    #Block3
    block3_sepconv1_act = layers.Activation('relu', name='block3_sepconv1_act')(block2_add)
    block3_sepconv1 = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv1')(block3_sepconv1_act)#63x63
    block3_sepconv1_bn = layers.BatchNormalization(axis=channel_axis, 
                                                   name='block3_sepconv1_bn')(block3_sepconv1)
    block3_sepconv2_act = layers.Activation('relu', 
                                            name='block3_sepconv2_act')(block3_sepconv1_bn)
    block3_sepconv2 = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv2')(block3_sepconv2_act)
    block3_sepconv2_bn = layers.BatchNormalization(axis=channel_axis, 
                                                   name='block3_sepconv2_bn')(block3_sepconv2)#63x63

    block3_pool = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block3_pool')(block3_sepconv2_bn)#32x32
    block3_add = layers.add([block3_pool, block2_residual2])

    block3_residual1 = layers.Conv2D(728, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False)(block3_add)#16x16
    block3_residual2 = layers.BatchNormalization(axis=channel_axis)(block3_residual1)

    #Block4
    block4_sepconv1_act = layers.Activation('relu', name='block4_sepconv1_act')(block3_add)
    block4_sepconv1 = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv1')(block4_sepconv1_act)#32x32
    block4_sepconv1_bn = layers.BatchNormalization(axis=channel_axis, name='block4_sepconv1_bn')(block4_sepconv1)
    block4_sepconv2_act = layers.Activation('relu', name='block4_sepconv2_act')(block4_sepconv1_bn)
    block4_sepconv2 = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv2')(block4_sepconv2_act)
    block4_sepconv2_bn = layers.BatchNormalization(axis=channel_axis, name='block4_sepconv2_bn')(block4_sepconv2)

    block4_pool = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block4_pool')(block4_sepconv2_bn) #16x16
    block4_add = layers.add([block4_pool, block3_residual2])

    x = block4_add
    
    #MiddleBlock
        
    for i in range(8):
        #Always 16x16 inside middle block
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv1')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv1_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv2')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv2_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv3')(x)
        x = layers.BatchNormalization(axis=channel_axis,
                                      name=prefix + '_sepconv3_bn')(x)
        x = layers.add([x, residual])

    middle_block_last_layer = x
    
    
    middle_residual1 = layers.Conv2D(1024, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False)(middle_block_last_layer) #8x8
    middle_residual2 = layers.BatchNormalization(axis=channel_axis)(middle_residual1)

    #Exit block1
    block13_sepconv1_act = layers.Activation('relu', name='block13_sepconv1_act')(middle_block_last_layer)
    block13_sepconv1 = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv1')(block13_sepconv1_act) #16x16
    block13_sepconv1_bn = layers.BatchNormalization(axis=channel_axis, name='block13_sepconv1_bn')(block13_sepconv1)
    block13_sepconv2_act = layers.Activation('relu', name='block13_sepconv2_act')(block13_sepconv1_bn)
    block13_sepconv2 = layers.SeparableConv2D(1024, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv2')(block13_sepconv2_act)
    block13_sepconv2_bn = layers.BatchNormalization(axis=channel_axis, name='block13_sepconv2_bn')(block13_sepconv2)

    block13_pool = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool')(block13_sepconv2_bn) #8x8
    block13_add = layers.add([block13_pool, middle_residual2])

    block14_sepconv1 = layers.SeparableConv2D(1536, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv1')(block13_add) #8x8
    block14_sepconv1_bn = layers.BatchNormalization(axis=channel_axis, name='block14_sepconv1_bn')(block13_add)
    block14_sepconv1_act = layers.Activation('relu', name='block14_sepconv1_act')(block14_sepconv1_bn)

    block14_sepconv2 = layers.SeparableConv2D(2048, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv2')(block14_sepconv1_act)
    block14_sepconv2_bn = layers.BatchNormalization(axis=channel_axis, name='block14_sepconv2_bn')(block14_sepconv2)
    block14_sepconv2_act = layers.Activation('relu', name='block14_sepconv2_act')(block14_sepconv2_bn) #8x8
    
    start_f = 512
    
    #Decoder
    start_f = start_f // 2 #256
    up1 = tf.keras.layers.UpSampling2D(2, interpolation='bilinear', name='up1')(block14_sepconv2_act) #16x16
    conc1 = tf.keras.layers.concatenate([block13_sepconv2_bn, up1], axis=-1, name='conc1')
    conv3 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv3')(conc1)
    conv4 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv4')(conv3)
    drop1 = tf.keras.layers.Dropout(0.1, seed=SEED)(conv4)
    
    start_f = start_f // 2 #128
    up2 = tf.keras.layers.UpSampling2D(2, interpolation='bilinear', name='up2')(drop1) #32x32
    conc2 = tf.keras.layers.concatenate([block4_sepconv2_bn, up2], axis=-1, name='conc2')
    conv5 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv5')(conc2)
    conv6 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv6')(conv5)
    drop2 = tf.keras.layers.Dropout(0.1, seed=SEED)(conv6)
    
    start_f = start_f // 2 #64
    up3 = tf.keras.layers.UpSampling2D(2, interpolation='bilinear', name='up3')(drop2) #64x64
    zero_pad3 = tf.keras.layers.ZeroPadding2D(((0,1),(0,1)))(block3_sepconv2_bn)
    conc3 = tf.keras.layers.concatenate([zero_pad3, up3], axis=-1, name='conc3')
    conv7 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv7')(conc3)
    conv8 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv8')(conv7)
    drop3 = tf.keras.layers.Dropout(0.1, seed=SEED)(conv8)
    
    start_f = start_f // 2 #32
    up4 = tf.keras.layers.UpSampling2D(2, interpolation='bilinear', name='up4')(drop3) #128x128
    zero_pad4 = tf.keras.layers.ZeroPadding2D(((1,2),(1,2)))(block2_sepconv2_bn)
    conc4 = tf.keras.layers.concatenate([zero_pad4, up4], axis=-1, name='conc4')
    conv9 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv9')(conc4)
    conv10 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv10')(conv9)
    drop4 = tf.keras.layers.Dropout(0.1, seed=SEED)(conv10)
    
    start_f = start_f // 2 #16
    up5 = tf.keras.layers.UpSampling2D(2, interpolation='bilinear', name='up5')(drop4)
    conv11 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv11')(up5)
    conv12 = tf.keras.layers.Conv2D(filters=start_f,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 strides=(1, 1),
                                 padding='same',
                                 name='conv12')(conv11)
    
    #Output
    outputs = tf.keras.layers.Conv2D(filters=1,
                                 kernel_size=(1, 1),
                                 strides=(1, 1),
                                 padding='same',
                                 activation='sigmoid',
                                 name='out1')(conv12)
    
    
    model = tf.keras.models.Model(inputs=[img_input], outputs=[outputs])
    
    weights_path = tf.keras.utils.get_file(
        'xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
        TF_WEIGHTS_PATH_NO_TOP,
        cache_subdir='models',
        file_hash='b0042744bf5b25fce3cb969f33bebb97')
    
    model.load_weights(weights_path, by_name=True)
    
    return model

In [ ]:
model = create_model()

for layer in model.layers[:11]:
    layer.trainable = False
for layer in model.layers[11:]:
    layer.trainable = True
    
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

In [ ]:
# Optimization params
# -------------------

# Loss

def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    return 1 - (numerator + 1) / (denominator + 1)

def combined_loss(y_true, y_pred):
    def dice_loss1(y_true, y_pred):
        numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
        denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
        return tf.reshape(1 - numerator / denominator, (-1, 1, 1))

    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss1(y_true, y_pred)

# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# learning rate
lr = 1e-3
sgd = tf.keras.optimizers.SGD(lr=lr, decay=1e-5, momentum=0.5, nesterov=True)
sparse = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
# -------------------

# Validation metrics
# ------------------
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

# ------------------

# Compile Model
model.compile(optimizer=sgd, loss=[combined_loss], metrics=[my_IoU])

Training with callbacks
---

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'UNet'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                   save_weights_only=True,
                                                   monitor='val_my_IoU',
                                                   save_best_only=True,
                                                   mode='max')  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


history = model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.savefig('loss.png')

plt.figure(2)
plt.plot(history.history['my_IoU'])
plt.plot(history.history['val_my_IoU'])
plt.title('Intersection over Union')
plt.ylabel('IoU')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.savefig('my_IoU.png')
plt.show()

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Compute Prediction
---

In [ ]:
import time
import matplotlib.pyplot as plt

from PIL import Image

%matplotlib notebook

# Cycle over test images
test_dir = os.path.join(dataset_dir, 'test')
test_img_dir = os.path.join(test_dir, 'images', 'img')

CSV File Creation
---

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')

        for key, value in results.items():
            value
            f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

Image Flatten Codification
----

In [ ]:
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels
    pixels = np.concatenate([[0], pixels, [0]])
    pixels
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs
    runs[1::2] -= runs[::2]
    runs
    return ' '.join(str(x) for x in runs)

Model Predictions
---

In [ ]:
from PIL import Image
import numpy as np

img_filenames = next(os.walk(test_img_dir))[2]

#model.load_weights(os.path.join(cwd, 'segmentation_experiments/UNet_Dec13_16-49-02/ckpts/cp_16.ckpt'))

fig, ax = plt.subplots(1, 2, figsize=(8, 8))
fig.show()

results={}
i=1
for img_filename in img_filenames:
    
    
    img = Image.open(os.path.join(test_img_dir, img_filename))
    img = img.resize((256, 256))
    
    img_arr = np.expand_dims(np.array(img), 0)
    img_arr = preprocess_input(img_arr)
    out_sigmoid = model.predict(x=img_arr)
    
    out_sigmoid = tf.cast(out_sigmoid > 0.5, tf.int32)
    out_sigmoid = out_sigmoid[0]
    print(out_sigmoid)
    # Get predicted class as the index corresponding to the maximum value in the vector probability
    # Assign colors (just for visualization)
    
    prediction_gray = np.zeros([256, 256, 1])
    #prediction_rgb = np.zeros([256, 256, 3])
    
    prediction_gray[np.where(out_sigmoid == 0)] = [0]
    prediction_gray[np.where(out_sigmoid == 1)] = [1]
    
    #prediction_rgb[np.where(out_sigmoid == 0)] = [0, 0, 0]
    #prediction_rgb[np.where(out_sigmoid == 1)] = [255, 255, 255]
    
    prediction_rgb = tf.where(out_sigmoid > 0, 
                             tf.constant(255, tf.int32, [256, 256, 3]),  #buildings
                             tf.constant(0, tf.int32, [256, 256, 3]))    #background
    
    #prediction_gray = tf.where(out_sigmoid > 0,
    #                           tf.constant(1, tf.int32, [256, 256, 1]),  #buildings
    #                           tf.constant(0, tf.int32, [256, 256, 1]))  #background

    
    runs = rle_encode(prediction_gray)
    results[img_filename[:-4]] = runs
    print(i)
    i+=1
    
    ax[0].imshow(np.uint8(img_arr[0, ...]))
    ax[1].imshow(np.uint8(prediction_rgb))
    
    fig.canvas.draw()
    #time.sleep(10)

create_csv(results)

In [ ]:
import shutil
shutil.make_archive('training', 'zip', 'segmentation_experiments')